Adapted from https://developers.google.com/optimization/assignment/assignment_cp#mip_solution

### Import libraries

In [ ]:
from ortools.linear_solver import pywraplp

### Define data

In [ ]:
# List of tasks to be assigned
# cat_a and cat_b are binary flags that classify each task on some dimension
# They're integers so they can be summed to compare to a worker's allowable range of tasks
# having a given category
tasks = [
    {
        "id": 1,
        "profit": 12,
        "cat_a": 0,
        "cat_b": 1,
    },
    {
        "id": 2,
        "profit": 32,
        "cat_a": 0,
        "cat_b": 0,
    },
    {
        "id": 3,
        "profit": 54,
        "cat_a": 1,
        "cat_b": 1,
    },
    {
        "id": 4,
        "profit": 5,
        "cat_a": 1,
        "cat_b": 0,
    },
    {
        "id": 5,
        "profit": 11,
        "cat_a": 1,
        "cat_b": 1,
    },
    {
        "id": 6,
        "profit": 45,
        "cat_a": 0,
        "cat_b": 1,
    },
]

workers = [
    {
        "id": 1,
        "max_a": 2,
        "max_b": 100,  # arbitrarily large value
    },
    {
        "id": 2,
        "max_a": 4,
        "max_b": 100,
    },
    {
        "id": 3,
        "max_a": 2,
        "max_b": 100,
    },
]

In [ ]:
l = [task.get("cat_a") for task in tasks]
print(l)

### Declare solver

In [ ]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

### Create variables

In [ ]:
# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for worker in workers:
    for task in tasks:
        x[worker["id"], task["id"]] = solver.BoolVar(f'x[{worker["id"]},{task["id"]}]')

### Add constraints

In [ ]:
# The sum of task Category A (cat_a) values is <= the worker's Maximum A (max_a) value
for worker in workers:
    solver.Add(
        solver.Sum([
            task.get("cat_a") * x[worker.get("id"), task.get("id")] for task in tasks
        ]) <= worker.get("max_a")
        )

# Each task is assigned to exactly one worker.
for task in tasks:
    solver.Add(
        solver.Sum([x[worker.get("id"), task.get("id")] for worker in workers]) == 1)

### Create objective

In [ ]:
objective_terms = []
for worker in workers:
    for task in tasks:
        objective_terms.append(task.get("profit") * x[worker.get("id"), task.get("id")])

solver.Maximize(solver.Sum(objective_terms))

### Invoke the solver

In [ ]:
status = solver.Solve()

### Display results

In [ ]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total profit = {solver.Objective().Value()}\n')
    for worker in workers:
        for task in tasks:
            if x[worker.get("id"), task.get("id")].solution_value() > 0.5:
                print(f'Worker {worker.get("id")} assigned to task {task.get("id")}.'
                      + f' Profit: {task.get("profit")}'
                      )
else:
    print('No solution found.')